# Importing Libraries


In [ ]:
#IT19178400_Ranasinghe R.A.P.T. #IT19119304_Perera J.K.H.A.K. #IT19181820_Akash Y.A.K. #IT19130576_Nanayakkara K.O.K.L.

#Startup Success Prediction Using Random Forest Algorithm

#Perform mathematical and statistical operations 
import numpy as np

#Read data files and create data frames
import pandas as pd

#Import related to visualization
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
#Split data in to train set and test set
from sklearn.model_selection import train_test_split

In [ ]:
#Import random forest algorithm
from sklearn.ensemble import RandomForestClassifier

In [ ]:
#Import classification report 
from sklearn.metrics import classification_report

In [ ]:
#Import of accuracy score
import sklearn.metrics as sm

In [ ]:
#Import recall score and percision score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

# Importing & Loading the dataset

In [ ]:
#Get data from the churn.csv file to the bank_data dataframe
startup_dataset = pd.read_csv('startup.csv')

In [ ]:
startup_dataset

In [ ]:
#Get an idea about the dataset
startup_dataset.head()

# Dataset Info:

In [ ]:
#Check information related to the dataframe
startup_dataset.info()

In [ ]:
#Check whether if there are any null values
startup_dataset.isnull()

In [ ]:
sum(startup_dataset.isnull().sum())

In [ ]:
#Display Missing values in a grid view
nullValue=pd.DataFrame(startup_dataset.isnull().sum(),columns = ["Null Values"])
nullValue["Missing Values (%)"]=(startup_dataset.isna().sum() / len(startup_dataset)*100)
nullValue = nullValue[nullValue["Missing Values (%)"]  >  0]
nullValue.style.background_gradient(cmap='magma',low =0.3,high=0.2) 

In [ ]:
#Handling Missing Values in 'Unnamed: 6', 'closed_at', 'age_first_milestone_year', 'age_last_milestone_year' columns
startup_dataset['Unnamed: 6'] = startup_dataset.apply(lambda row: (row.city) + " " + (row.state_code) + " " +(row.zip_code)  , axis = 1)

startup_dataset['closed_at'] = startup_dataset['closed_at'].fillna(value="31/12/2013")

startup_dataset['age_first_milestone_year'] = startup_dataset['age_first_milestone_year'].fillna(value="0")

startup_dataset['age_last_milestone_year'] = startup_dataset['age_last_milestone_year'].fillna(value="0")

In [ ]:
#Handling Missing Values in 'state_code.1' column
for index, row in startup_dataset.iterrows():
    if row['state_code']!=row['state_code.1']:
        print(index, row['state_code'], row['state_code.1'])
        
startup_dataset.drop(["state_code.1"], axis=1, inplace=True)

In [ ]:
#Graphical view of null values in the whole dataframe
sns.heatmap(startup_dataset.isnull(),yticklabels=False, cbar=False, cmap='viridis')

In [ ]:
#Numerical value of the total number of null values in the data set
total_null_values = sum(startup_dataset.isnull().sum())
print(total_null_values)